In [459]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, roc_auc_score,accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
le = LabelEncoder()
out_transform = LabelEncoder()

In [460]:
train = pd.read_csv('train_ctrUa4K.csv',sep=',')
train['Loan_Status'] = out_transform.fit_transform(train['Loan_Status'])
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [461]:
test = pd.read_csv('test_lAUu6dG.csv',sep=',')
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [462]:
test['Loan_Status'] = -1
# for column in train.columns:
#     print(column,train[column].nunique(),test[column].nunique())

In [463]:
df_all = pd.concat([train,test])
del df_all['Loan_ID']
df_all.reset_index(drop=True,inplace=True)
df_all.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [464]:
df_all.isnull().sum()

Gender               24
Married               3
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [465]:
# important
train.loc[train['Credit_History']==0,'Loan_Status'].value_counts()

0    82
1     7
Name: Loan_Status, dtype: int64

In [466]:
mmode = ['Gender', 'Education', 'Married', 'Self_Employed', 'Property_Area','Dependents','LoanAmount']
mmean = ['Loan_Amount_Term']
def na_filler(df):
    df['Credit_History'].fillna(1,inplace=True)
    for col in mmode:
        df[col].fillna(df[col].mode()[0],inplace=True)
    for col in mmean:
        df[col].fillna(df[col].mean(),inplace=True)

na_filler(df_all)

In [467]:
df_all.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [468]:
cat_cols = ['Gender',
            'Married',
            'Education',
            'Self_Employed',
            'Property_Area',
            'Dependents']
df_all[cat_cols] = df_all[cat_cols].apply(lambda col: le.fit_transform(col))
df_all.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849,0.0,120.0,360.0,1.0,2,1
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1


In [469]:
#feature generation.
df_all['CombinedIncome'] = df_all['ApplicantIncome']+df_all['CoapplicantIncome']
df_all['Loan_Income_Ratio'] = (df_all.loc[df_all['LoanAmount']>=0,'LoanAmount']*1000)/df_all.loc[df_all['LoanAmount']>=0,'CombinedIncome'] 
df_all['Loan_Per_Month'] = (df_all.loc[(df_all['LoanAmount']>=0)&(df_all['Loan_Amount_Term']>=0),'LoanAmount']*1000)/df_all.loc[(df_all['LoanAmount']>=0)&(df_all['Loan_Amount_Term']>=0),'Loan_Amount_Term']
df_all['Loan_Income_Ratio'].fillna(-1,inplace=True)
df_all['Loan_Per_Month'].fillna(-1,inplace=True)
df_all.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,CombinedIncome,Loan_Income_Ratio,Loan_Per_Month
0,1,0,0,0,0,5849,0.0,120.0,360.0,1.0,2,1,5849.0,20.516328,333.333333
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0,6091.0,21.014612,355.555556
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1,3000.0,22.000000,183.333333
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1,4941.0,24.286582,333.333333
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1,6000.0,23.500000,391.666667


In [470]:
train = df_all[df_all['Loan_Status']!=-1]
train = train[train['Credit_History']==1]
train.reset_index(inplace=True,drop=True)
train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,CombinedIncome,Loan_Income_Ratio,Loan_Per_Month
0,1,0,0,0,0,5849,0.0,120.0,360.0,1.0,2,1,5849.0,20.516328,333.333333
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0,6091.0,21.014612,355.555556
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1,3000.0,22.000000,183.333333
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1,4941.0,24.286582,333.333333
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1,6000.0,23.500000,391.666667


In [471]:
test = df_all[df_all['Loan_Status']==-1]
test.reset_index(inplace=True,drop=True)
del test['Loan_Status']
test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,CombinedIncome,Loan_Income_Ratio,Loan_Per_Month
0,1,1,0,0,0,5720,0.0,110.0,360.0,1.0,2,5720.0,19.230769,305.555556
1,1,1,1,0,0,3076,1500.0,126.0,360.0,1.0,2,4576.0,27.534965,350.000000
2,1,1,2,0,0,5000,1800.0,208.0,360.0,1.0,2,6800.0,30.588235,577.777778
3,1,1,2,0,0,2340,2546.0,100.0,360.0,1.0,2,4886.0,20.466639,277.777778
4,1,0,0,1,0,3276,0.0,78.0,360.0,1.0,2,3276.0,23.809524,216.666667


In [472]:
y = train[['Loan_Status']]
del train['Loan_Status']
X = train
# X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.2, random_state=1,shuffle=True,stratify=y)

In [473]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
accxgb = []
y_pred_totxgb = []

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123456789)

for train_index, test_index in fold.split(X,y):
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    alg = XGBClassifier(learning_rate=0.01, n_estimators=140, max_depth=5,
                        objective='binary:logistic', nthread=4, seed=1)
    alg.fit(X_train, y_train)
    y_pred = alg.predict(X_test)
    print("Accuracy: ", (round(accuracy_score(y_test, y_pred),2)))
    
    accxgb.append(round(accuracy_score(y_test, y_pred),2))
    p = alg.predict(test)
    y_pred_totxgb.append(p)

# Accuracy:  0.83
# Accuracy:  0.83
# Accuracy:  0.79
# Accuracy:  0.77
# Accuracy:  0.78

C:\Users\user\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy:  0.79
Accuracy:  0.79
Accuracy:  0.79
Accuracy:  0.84
Accuracy:  0.79


In [474]:
# np.mean(y_pred_totxgb,0)
df = pd.DataFrame(y_pred_totxgb)
df = df.T
df.head()

,0,1,2,3,4
0,1,1,1,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1


In [475]:
df = df.mode(axis=1)
df.head()

,0
0,1
1,1
2,1
3,1
4,1


In [476]:
y_test = out_transform.inverse_transform(df)
y_test

C:\Users\user\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [477]:
test['Loan_Status'] = y_test
test.loc[test['Credit_History']==0,'Loan_Status']='N'

C:\Users\user\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [478]:
submission = pd.read_csv('sample_submission_49d68Cx.csv')
submission.head()

,Loan_ID,Loan_Status
0,LP001015,N
1,LP001022,N
2,LP001031,N
3,LP001035,N
4,LP001051,N


In [479]:
submission['Loan_Status'] = test['Loan_Status']
submission.head()

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y


In [480]:
submission.to_csv('submission.csv',sep=',',index=False)
